In [113]:
import numpy as np
import scipy as sp
import math
from sklearn import datasets, base
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import pandas as pd
from sklearn.cross_validation import cross_val_score
from copy import deepcopy
classifiers = [
    KNeighborsClassifier(n_jobs = -1),
    SVC(probability = True),
#     GaussianProcessClassifier(n_jobs = -1),
    DecisionTreeClassifier(),
    RandomForestClassifier()]

# classifiers_name = ["KNN", "SVM", "GP", "DT", "RF"]
classifiers_name = ["KNN", "SVM", "DT", "RF"]
beta = "XGB"

parameters = {"KNN":{"n_neighbors":[]},
              "SVM":{"C":[],"kernel":[],"degree":[]},
              "GP":{"kernel":[],"n_restarts_optimizer":[]},
              "DT":{"splitter":[], "max_depth":[]},
              "RF":{"n_estimators":[], "max_depth":[] },
              "XGB":{"learning_rate":[], "n_estimators":[], "max_depth":[],"min_child_weight":[]}
            }
classifier_parameters = []

iris = datasets.load_iris()
X = iris.data
Y = iris.target
Df = pd.DataFrame()
Df["X"] = X.tolist()
Df["Y"] = Y.tolist()

In [114]:
def sample_hp_Ai(Ai):
    global classifier_parameters
    if(Ai is "KNN"):
        parameters["KNN"]["n_neighbors"] = math.ceil(np.random.uniform(100))
        classifier_parameters.append(parameters["KNN"].copy())
        return(parameters["KNN"])
    elif(Ai is "SVM"):
        parameters["SVM"]["C"] = float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0))))
        parameters["SVM"]["kernel"] = str(np.random.choice(["linear", "rbf"]))
        parameters["SVM"]["degree"] = np.random.randint(1,4)
        classifier_parameters.append(parameters["SVM"].copy())
        return(parameters["SVM"])
    elif(Ai is "GP"):
        parameters["GP"]["kernel"] = str(np.random.choice(["linear", "rbf"]))
        parameters["GP"]["n_restarts_optimizer"] = np.random.randint(1,10)
        classifier_parameters.append(parameters["GP"].copy())
        return(parameters["GP"])
    elif(Ai is "DT"):
        parameters["DT"]["splitter"] = np.random.choice(["best", "random"])
        parameters["DT"]["max_depth"] = np.random.randint(1,100)
        classifier_parameters.append(parameters["DT"].copy())
        return(parameters["DT"])
    elif(Ai is "RF"):
        parameters["RF"]["n_estimators"] = np.random.randint(1,20)
        parameters["RF"]["max_depth"] = np.random.randint(1,100)
        classifier_parameters.append(parameters["RF"].copy())
        return(parameters["RF"])
    elif(Ai is "XGB"):
        parameters["XGB"]["learning_rate"].append(float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0)))))
        parameters["XGB"]["n_estimators"].append(np.random.randint(800,2000))
        parameters["XGB"]["max_depth"].append(np.random.randint(5,200))
        parameters["XGB"]["min_child_weight"].append(np.random.randint(1,10))
        classifier_parameters.append(parameters["XGB"].copy())
        return(parameters["XGB"])

In [115]:
def get_params(A, B, P, N):
    mu = np.random.dirichlet(P)
    N = [int(mu_i*N) for mu_i in mu]
    [sample_hp_Ai(classifiers_name[i]) for i in range(len(classifiers_name)) for j in range(N[i])]
    Phi = sample_hp_Ai("XGB")
    Psi = [parameters[key] for key in parameters if key is not "XGB"]
    return Phi, N, Psi

In [116]:
Phi, N, Psi = get_params(classifiers_name, "RF", P=(np.arange(1,len(classifiers_name)+1)*100)[::-1], N=10)

In [117]:
N,classifier_parameters

([4, 2, 1, 1],
 [{'n_neighbors': 27},
  {'n_neighbors': 84},
  {'n_neighbors': 14},
  {'n_neighbors': 58},
  {'C': 1.06, 'degree': 1, 'kernel': 'rbf'},
  {'C': 0.17, 'degree': 1, 'kernel': 'linear'},
  {'max_depth': 14, 'splitter': 'random'},
  {'max_depth': 79, 'n_estimators': 15},
  {'learning_rate': [2.19],
   'max_depth': [162],
   'min_child_weight': [1],
   'n_estimators': [1171]}])

In [118]:
class Blend:
    def __init__(self, A = ["KNN", "SVM", "GP", "DT", "RF"], B = "XGB",D = [], L = 2, Phi = [], Psi = [], N=100 ):
        self.A = A
        self.B = B
        self.D = D
        self.L = L
        self.Phi = Phi
        self.Psi = Psi
        self.N = N
                    
    def fit(self, A, B, D, L, Phi, N, Psi):
#         p = np.random.normal(loc=0.7, scale=0.1)
        p=0.7
        Dfw = []
        M = []
        F = []
        G = []
        for l in range(1,L+1):
            D_dash = D.sample(frac=p)
            orig_index = sorted(Df.index.tolist())
            train_index = sorted(D_dash.index.tolist())
            test_index = sorted(set(orig_index) - set(train_index))
            D_dash_bar = Df.ix[test_index]
            classifier_index = 0
            for i in range(0,len(classifiers_name)):
                temp = []
                if(N[i]):
                    for j in range(0,N[i]):
                        print("Training: ", classifiers_name[i])
                        temp_clf = None
                        temp_clf = classifiers[i].set_params(**classifier_parameters[classifier_index])
#                         print("Fitting: {0}\n".format(temp_clf))
                        temp_clf = temp_clf.fit(X = D_dash["X"].tolist(), y = D_dash["Y"].tolist())
#                         print("Temp Result is: ",temp_clf.predict_proba([[4.6, 3.4, 1.4, 0.3]]))
                        temp.append(deepcopy(temp_clf))
                        classifier_index += 1
                M.append(temp)
#             for row in D_dash["X"]:
#                 for model in M[0]:
#                     print("MOdel is:",model)
#                     F.append(row*model.predict_proba([row]))
        return(M,D_dash,F)
            

In [119]:
def blending_ensemble(A, B, D, P, N, k, R):
    L = 2
    M = []
    for r in range(1,R+1):
        Phi_r, N_r, Psi_r = get_params(A, B, P, N=10)
        eps_r = cross_val_score(Blend(), A, B, D, L, Phi, Psi, N, cv=k)
        print(eps_r)

In [120]:
classifier_parameters

[{'n_neighbors': 27},
 {'n_neighbors': 84},
 {'n_neighbors': 14},
 {'n_neighbors': 58},
 {'C': 1.06, 'degree': 1, 'kernel': 'rbf'},
 {'C': 0.17, 'degree': 1, 'kernel': 'linear'},
 {'max_depth': 14, 'splitter': 'random'},
 {'max_depth': 79, 'n_estimators': 15},
 {'learning_rate': [2.19],
  'max_depth': [162],
  'min_child_weight': [1],
  'n_estimators': [1171]}]

In [121]:
b =Blend()

In [122]:
M_temp, D_dash, F = b.fit(A=classifiers_name, B="XGB", D=Df, L=1, Phi=Phi, N=N, Psi=Psi)

Training:  KNN
Training:  KNN
Training:  KNN
Training:  KNN
Training:  SVM
Training:  SVM
Training:  DT
Training:  RF


/home/neo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
